In [3]:
import requests
from bs4 import BeautifulSoup
import math
import csv

from tqdm import tqdm

# Target URL format with placeholder for pagination
target_url = 'https://www.linkedin.com/jobs/search/?currentJobId=4231638091&distance=25&geoId=90000724&keywords=senior%20data%20scientist&origin=JOBS_HOME_KEYWORD_HISTORY&refresh=true'

# List to store job IDs
job_ids = []
job_data = []

# Calculate number of pages to scrape (25 jobs per page)
total_jobs = 100  # Example number
pages = math.ceil(total_jobs/25)

# Get all job IDs
for i in tqdm(range(0, pages)):
    res = requests.get(target_url.format('Python', 'New York', i*25))
    soup = BeautifulSoup(res.text, 'html.parser')
    jobs_on_page = soup.find_all("li")

    for job in jobs_on_page:
        try:
            job_id = job.find("div", {"class": "base-card"}).get('data-entity-urn').split(":")[-1]
            job_ids.append(job_id)
        except:
            continue

# Get details for each job
job_details_url = 'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}'
for job_id in tqdm(job_ids):
    resp = requests.get(job_details_url.format(job_id))
    soup = BeautifulSoup(resp.text, 'html.parser')

    job_info = {}
    try:
        job_info["company"] = soup.find("div", {"class": "top-card-layout__card"}).find("a").find("img").get('alt')
        job_info["title"] = soup.find("div", {"class": "top-card-layout__entity-info"}).find("a").text.strip()
        job_info["level"] = soup.find("ul", {"class": "description__job-criteria-list"}).find("li").text.replace("Seniority level", "").strip()
        job_data.append(job_info)
    except:
        continue

# Save to CSV
with open('linkedin_jobs.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=["company", "title", "level"])
    writer.writeheader()
    writer.writerows(job_data)

100%|██████████| 104/104 [00:41<00:00,  2.50it/s]
